In [1]:
from datetime import datetime as dt
from collections import defaultdict
import time
import numpy as np
import pandas as pd
import math
from scipy import stats
from scipy.signal import argrelextrema
from scipy.signal import savgol_filter
import statsmodels.api as sm
from sympy import *
import pytz
import itertools
import matplotlib.pyplot as plt
from itertools import combinations
from itertools import product
from itertools import groupby
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import MetaTrader5 as mt5

In [2]:
if not mt5.initialize(path="C:/MetaTrader_A/MetaTrader 5/terminal64.exe",login=1049732, server="TradersWay-Demo",password="r5AH8znU"):
    print("initialize() failed, error code =",mt5.last_error())
    quit()

In [3]:
def positioner(a,b,c,d,e):
    
    riskusd = 50.00        
    symbol_info = mt5.symbol_info(a)
    point = mt5.symbol_info(a).point
    base, quote = a[:3], a[3:]
    
    ro = 2

    if quote == "USD":
        exchange = 1.00
        if base == "XAG":
            stpips = abs(e - c)*10000
            lot = round(((riskusd*exchange)/10)/1/stpips,ro)
        elif base == "XAU":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "BTC":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "BCH":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "ETH":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "DSH":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "LTC":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "XMR":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "ZEC":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
        elif base == "ETC":
            stpips = abs(e - c)*1000
            lot = round(((riskusd*exchange)/10)/10/stpips,ro)
        elif base == "XRP":
            stpips = abs(e - c)*10000
            lot = round(((riskusd*exchange)/10)/1/stpips,ro)
        else:
            stpips = abs(e - c)*10000
            lot = round(((riskusd*exchange)/10)/stpips,ro)
    if quote == "USDt":
        exchange = 1.00
        if base == "BTC":
            stpips = abs(e - c)*100
            lot = round(((riskusd*exchange)/10)/100/stpips,ro)
    elif quote == "AUD":
        exchange = 1/(mt5.symbol_info("AUDUSD").bid)
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == "CAD":
        exchange = mt5.symbol_info("USDCAD").bid
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == "CHF":
        exchange = mt5.symbol_info("USDCHF").bid
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == "EUR":
        exchange = 1/(mt5.symbol_info("EURUSD").bid)
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == "GBP":
        exchange = 1/(mt5.symbol_info("GBPUSD").bid)
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == "JPY":
        exchange = mt5.symbol_info("USDJPY").bid
        stpips = abs(e - c)*100
        lot = round(((riskusd*exchange)/10)/100/stpips,ro)
    elif quote == "NZD":
        exchange = 1/(mt5.symbol_info("NZDUSD").bid)
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == 'NOK':
        exchange = mt5.symbol_info("USDNOK").bid
        stpips = abs(e - c)*1000
        lot = round(((riskusd*exchange)/10)/10/stpips,ro)
    elif quote == 'SEK':
        exchange = mt5.symbol_info("USDSEK").bid
        stpips = abs(e - c)*1000
        lot = round(((riskusd*exchange)/10)/10/stpips,ro)
    elif quote == 'SGD':
        exchange = mt5.symbol_info("USDSGD").bid
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
    elif quote == 'DKK':
        exchange = mt5.symbol_info("USDDKK").bid
        stpips = abs(e - c)*1000
        lot = round(((riskusd*exchange)/10)/10/stpips,ro)
    elif quote == 'MXN':
        exchange = mt5.symbol_info("USDMXN").bid
        stpips = abs(e - c)*1000
        lot = round(((riskusd*exchange)/10)/10/stpips,ro)
    elif quote == "entCrude":
        exchange = 1.00
        stpips = abs(e - c)*10
        lot = round(((riskusd*exchange)/10)/1000/stpips,ro)
    elif quote == "ICrude":
        exchange = 1.00
        stpips = abs(e - c)*10
        lot = round(((riskusd*exchange)/10)/1000/stpips,ro)
    elif quote == "TRY":
        exchange = mt5.symbol_info("USDTRY").bid
        stpips = abs(e - c)*10000
        lot = round(((riskusd*exchange)/10)/stpips,ro)
        
    deviation = 20
    
    if b == "LONG":
        price = mt5.symbol_info_tick(a).ask
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": a,
            "volume": lot,
            "type": mt5.ORDER_TYPE_BUY,
            "price": price,
            "sl": e,
            "tp": d,
            "deviation": deviation,
            "magic": 234000,
            "comment": "PEPPER",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        selected=mt5.symbol_select(a,True)
        if not selected:
            print("Failed to select pair")
            mt5.shutdown()
            quit()
    elif b == "SHORT":
        price = mt5.symbol_info_tick(a).bid
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": a,
            "volume": lot,
            "type": mt5.ORDER_TYPE_SELL,
            "price": price,
            "sl": e,
            "tp": d,
            "deviation": deviation,
            "magic": 234000,
            "comment": "PEPPER",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }
        selected=mt5.symbol_select(a,True)
        if not selected:
            print("Failed to select pair")
            mt5.shutdown()
            quit()
            
    lotflag = "GREEN"
    if lot == 0:
        lotflag = "RED"
    
    print(a,"  ","LOT:  ",lot, "      STPIPS:   ",stpips,"  ",lotflag)

    return request, lotflag

In [4]:
def livepearls(h,i,j,k,l,m,J,K):

    w = l
    LPrates = mt5.copy_rates_from_pos(h, mt5.TIMEFRAME_M1, 0, 10)
    LP = LPrates
    currentprice = LP['close'][-1]
    retrace = 0
    if i == "LONG":
        if currentprice < J:
            retrace = 1
    if i == "SHORT":
        if currentprice > J:
            retrace = 1

    if retrace == 1:
        K += 1
        
        if K >= 7:
            microtrend = LP['close'][-1]-LP['open'][0]
            if i == "LONG":
                if microtrend > 0:
                    microdirection = 1
                else:
                    microdirection = 0
            else:
                if microtrend < 0:
                    microdirection = 1
                else:
                    microdirection = 0
            if microdirection == 1:
                scopetheta = abs(math.degrees(math.atan(microtrend)))
                if scopetheta > 0.5:
                    if i == "LONG":
                        if currentprice > l:
                            tpdist = k - j
                            w = j - tpdist
                    else:
                        if currentprice < l:
                            tpdist = j - k
                            w = j + tpdist
                    request, lotflag = positioner(h,i,j,k,l)
                    if lotflag == "GREEN":
                        result = mt5.order_send(request)
                        if result.retcode != mt5.TRADE_RETCODE_DONE:
                            print("order_send failed, retcode={}".format(result.retcode))
                            result_dict=result._asdict()
                            for field in result_dict.keys():
                                print("   {}={}".format(field,result_dict[field]))
                                if field=="request":
                                    traderequest_dict=result_dict[field]._asdict()
                                    for tradereq_filed in traderequest_dict:
                                        print("       traderequest: {}={}".format(tradereq_filed,traderequest_dict[tradereq_filed]))
                        K = "NO"
                    else:
                        K = "NO"
    
    return w, K

In [5]:
def levelgrab(pair):
    
    levslist = []
    
    for x in pair:
        Srates = mt5.copy_rates_from_pos(x, mt5.TIMEFRAME_H4, 0, 1000)
        S = Srates
        SCM = np.average(abs(S['close'] - S['open']))
        y = S['close']
        LVL = np.concatenate((y[argrelextrema(y, np.greater)[0]],y[argrelextrema(y, np.less)[0]]))
        B = list(combinations(LVL, 2))
        Ba = np.array([x for x, y in B])
        B = np.array([abs(x - y) for x, y in B])
        Bb = np.array([np.where(x <= SCM, 1, 0) for x in B])
        T = [(Ba[i], Bb[i]) for i in range(0, len(Ba))] 
        Toutput = defaultdict(int) 
        for k, v in T: 
            Toutput[k] += v 
        table = list(Toutput.items())
        tablevs = [x for x, y in table]
        tabpings = [y for x, y in table]
        tabmax = argrelextrema(np.array(tabpings), np.greater)
        tabmax2 = list(tabmax[0])
        levs = np.array(tablevs)[tabmax2]
        levs = list(levs)
        levs.sort(reverse=True)
        levs = list(levs)
        
        mindist = min([abs(j-i) for i, j in zip(levs[:-1], levs[1:])])
        
        while mindist < SCM:
            i = 0
            h = -1
            j = 1
            for c in range(0, len(levs)):
                if j == len(levs):
                    j = -1
                if abs(levs[i]-levs[h]) < SCM:
                    levs[i] = 0
                elif abs(levs[i]-levs[j]) < SCM:
                    levs[i] = levs[i] - abs(levs[i]-levs[j])/2
                i += 1
                h += 1
                j += 1
            levs = [v for v in levs if v != 0]
            mindist = min([abs(a-b) for a, b in zip(levs[:-1], levs[1:])])
            
        levs = tuple(levs)

        levslist.append(levs)

    return levslist


In [6]:
## WATCHLIST = ['AUD/CAD', 'AUD/CHF', 'AUD/JPY', 'AUD/NZD', 'AUD/USD', 'CAD/CHF', 'CAD/JPY', 'CHF/JPY', 'EUR/AUD', 'EUR/CAD', 'EUR/CHF', 'EUR/GBP', 'EUR/JPY', 'EUR/NZD', 'EUR/USD', 'GBP/AUD', 'GBP/CAD', 'GBP/CHF', 'GBP/JPY', 'GBP/NZD', 'GBP/USD', 'NZD/CAD', 'NZD/CHF', 'NZD/JPY', 'NZD/USD', 'USD/CAD', 'USD/CHF', 'USD/JPY', 'XAG/USD', 'XAU/USD', 'BCH/USD', 'BTC/USD', 'EOS/USD', 'ETH/USD', 'LTC/USD', 'XLM/USD', 'XRP/USD']

## FOREX = ## WATCHLIST = ['AUDCAD','AUDCHF','AUDJPY','AUDNZD','AUDUSD','CADCHF','CADJPY','CHFJPY','EURAUD','EURCAD','EURCHF','EURGBP','EURJPY','EURNZD','EURUSD','GBPAUD','GBPCAD','GBPCHF','GBPJPY','GBPUSD','NZDCAD','NZDCHF','NZDJPY','NZDUSD','USDCAD','USDCHF','USDJPY']

def brutus(pair,levslist,time_id):

    currentround = time_id
    
    ia = 0
    for x in pair:
        
        currency = x[3:]
        F = mt5.copy_rates_from_pos(x, mt5.TIMEFRAME_M15, 0, 200)
        x1 = np.linspace(1, 200, num=200)
        yclose = F['close']
        if currency == "JPY":
            y1 = yclose*100
        else:
            y1 = yclose*10000
        slope, intercept, r_value, p_value, std_err = stats.linregress(x1,y1)
        wholeslope = slope
        scopetheta = abs(math.degrees(math.atan(wholeslope)))
        
        if scopetheta > 5:

            DTrates = mt5.copy_rates_from_pos(x, mt5.TIMEFRAME_M5, 0, 168)
            DT = DTrates
            movetrend = np.average(DT['close'] - DT['open'])
            
            if wholeslope > 0:
                if movetrend > 0:
                    ditrend = 1
                elif movetrend < 0:
                    ditrend = -1
                else:
                    ditrend = 0
            elif wholeslope < 0:
                if movetrend < 0:
                    ditrend = 1
                elif movetrend > 0:
                    ditrend = -1
                else:
                    ditrend = 0
            else:
                ditrend = 0

            if ditrend == 1:
                
                levels = levslist[ia]

                Drates = mt5.copy_rates_from_pos(x, mt5.TIMEFRAME_M5, 0, 3)
                D = Drates
                DIRECT = np.where(D['close']-D['open'] > 0, 'Bullish', 'Bearish')
                CM = np.average(abs(D['close']-D['open']))
                PU = D[0]

                pOTESTLVL = 0
                for u in levels:
                    if abs(PU[1]-u) <= CM:
                        pOTESTLVL = u
                pCTESTLVL = 0
                for u in levels:
                    if abs(PU[4]-u) <= CM:
                        pCTESTLVL = u

                if DIRECT[0] == 'Bullish':
                    if movetrend > 0:
                        pdstrend = 1
                    elif movetrend < 0:
                        pdstrend = -1
                    else:
                        pdstrend = 0
                elif DIRECT[0] == 'Bearish':
                    if movetrend < 0:
                        pdstrend = 1
                    elif movetrend > 0:
                        pdstrend = -1
                    else:
                        pdstrend = 0
                else:
                    pdstrend = 0

                pcolor = 'white'
                if pCTESTLVL != 0:
                    if pOTESTLVL == pCTESTLVL:
                        pcolor = 'gray'
                    elif pOTESTLVL != pCTESTLVL:
                        if pdstrend == -1:
                            pcolor = 'purple'

                if pcolor == 'purple':
                    pcoltrade = 'TRADE'
                else:
                    pcoltrade = 0

                if pcoltrade == 'TRADE':
            
                    P = D[1]

                    OTESTLVL = 0
                    for u in levels:
                        if abs(P[1]-u) <= CM:
                            OTESTLVL = u
                    CTESTLVL = 0
                    for u in levels:
                        if abs(P[4]-u) <= CM:
                            CTESTLVL = u

                    BREAKLVL = 0
                    for u in levels:
                        if DIRECT[1] == 'Bullish':
                            if P[4] > (u+CM):
                                if P[1] < u:
                                    BREAKLVL = u
                                elif OTESTLVL == u:
                                    BREAKLVL = u
                        elif DIRECT[1] == 'Bearish':
                            if P[4] < (u-CM):
                                if P[1] > u:
                                    BREAKLVL = u
                                elif OTESTLVL == u:
                                    BREAKLVL = u

                    if DIRECT[1] == 'Bullish':
                        if movetrend > 0:
                            dstrend = 1
                        elif movetrend < 0:
                            dstrend = -1
                        else:
                            dstrend = 0
                    elif DIRECT[1] == 'Bearish':
                        if movetrend < 0:
                            dstrend = 1
                        elif movetrend > 0:
                            dstrend = -1
                        else:
                            dstrend = 0
                    else:
                        dstrend = 0

                    color = 'white'
                    if BREAKLVL != 0:
                        if CTESTLVL != BREAKLVL:
                            if dstrend == 1:
                                color = 'pearl'

                    if color == 'pearl':
                        coltrade = 'TRADE'
                    else:
                        coltrade = 0

                    if coltrade == 'TRADE':

                        Hrates = mt5.copy_rates_from_pos(x, mt5.TIMEFRAME_H2, 0, 25)
                        H = Hrates['close']
                        hlin = np.linspace(1,25,num=25)
                        H = savgol_filter(H, 13, 3)
                        SLOPE = []
                        i = 0
                        for t in hlin:
                            ii = i - 1
                            if i == 0:
                                SLOPE.append(0)
                            else:
                                slope = H[i] - H[ii]
                                SLOPE.append(slope)
                            i += 1
                        SLOPE2 = []
                        i = 0
                        for t in hlin:
                            ii = i - 1
                            if i == 0:
                                SLOPE2.append(0)
                            else:
                                slope2 = SLOPE[i] - SLOPE[ii]
                                SLOPE2.append(slope2)
                            i += 1
                        CONCAVE = []
                        i = 0
                        for t in hlin:
                            if SLOPE2[i] == 0:
                                CONCAVE.append('INFL')
                            elif SLOPE2[i] > 0:
                                CONCAVE.append('UP')
                            else:
                                CONCAVE.append('DOWN')
                            i += 1

                        AXCONCAVE = CONCAVE[-1]
                        if movetrend > 0:
                            if AXCONCAVE == 'UP':
                                contrade = 'TRADE'
                            else:
                                contrade = 0
                        elif movetrend < 0:
                            if AXCONCAVE == 'DOWN':
                                contrade = 'TRADE'
                            else:
                                contrade = 0
                        else:
                            contrade = 0

                        if contrade == 'TRADE':
                            
                            G = BREAKLVL

                            if movetrend > 0:
                                position = 'LONG'
                                entry = BREAKLVL + CM
                            elif movetrend < 0:
                                position = 'SHORT'
                                entry = BREAKLVL - CM
                            else:
                                position = 0
                                entry = 0

                            cend = int(len(levels)-1)
                            if position == 'LONG':
                                clindex = levels.index(BREAKLVL)
                                tpindex = int(clindex - 1)
                                slindex = int(clindex + 1)
                                if clindex == 0:
                                    stoploss = levels[slindex]
                                    sldist = entry - stoploss
                                    takeprofit = entry + sldist
                                elif clindex == cend:
                                    takeprofit = levels[tpindex]
                                    tpdist = takeprofit - entry
                                    stoploss = entry - tpdist
                                else:
                                    takeprofit = levels[tpindex]
                                    stoploss = levels[slindex]
                            elif position == 'SHORT':
                                clindex = levels.index(BREAKLVL)
                                tpindex = int(clindex + 1)
                                slindex = int(clindex - 1)
                                if clindex == 0:
                                    takeprofit = levels[tpindex]
                                    tpdist = entry - takeprofit
                                    stoploss = entry + tpdist
                                elif clindex == cend:
                                    stoploss = levels[slindex]
                                    sldist = stoploss - entry
                                    takeprofit = entry - sldist
                                else:
                                    takeprofit = levels[tpindex]
                                    stoploss = levels[slindex]
                            else:
                                takeprofit = 0
                                stoploss = 0

                            shrink = 0.90
                        if position == 'LONG':
                            tpdist = takeprofit - entry
                            takeprofit = entry + tpdist*shrink
                        elif position == 'SHORT':
                            tpdist = entry - takeprofit
                            takeprofit = entry - tpdist*shrink

                        if position == 'LONG':
                            tpdist = takeprofit - entry
                            sldist = entry - stoploss
                            if sldist > tpdist:
                                stoploss = entry - tpdist
                        elif position == 'SHORT':
                            tpdist = entry - takeprofit
                            sldist = stoploss - entry
                            if sldist > tpdist:
                                stoploss = entry + tpdist

                            prp = (x,position,entry,takeprofit,stoploss,CM,G)
                            gpurp = [(a,b,c,d,e,f,g) for (a,b,c,d,e,f,g,h) in purp]
                            gpflag = 0
                            for xg in gpurp:
                                if xg == prp:
                                    gpflag = 1
                            if gpflag != 1:
                                prp = (x,position,entry,takeprofit,stoploss,CM,G,currentround)
                                purp.append(prp)
    
        ia += 1
    
    purps = pd.DataFrame(purp, columns =['x','position','entry','takeprofit','stoploss','CM','G','time_id']) 
    purps['spy'] = 0
    purpcnt = int(purps['x'].count())
    purplin = np.linspace(1,purpcnt,num=purpcnt)
    print(purps)
    
    watchers = purps['spy']
    ite = 0
    while any(s != "NO" for s in watchers):
        i = 0
        for t in purplin:
            if purps.iloc[i,8] != "NO":
                timediff = currentround - purps.iloc[i,7]
                if timediff >= 60:
                    purps.iloc[i,8] = "NO"
                else:
                    Xpair = purps.iloc[i,0]
                    position = purps.iloc[i,1]
                    entry = purps.iloc[i,2]
                    takeprofit = purps.iloc[i,3]
                    stoploss = purps.iloc[i,4]
                    CM = purps.iloc[i,5]
                    k1 = purps.iloc[i,6]
                    k2 = purps.iloc[i,8]
                    W, spy = livepearls(Xpair,position,entry,takeprofit,stoploss,CM,k1,k2)
                    purps.iloc[i,4] = W
                    purps.iloc[i,8] = spy
            i += 1
        watchers = purps['spy']
        if ite == 28:
            break
        ite += 1
        time.sleep(10)
        
    purpdrop = [np.where(purps.spy=="NO",purps.index,"n")]
    purpdrop = [ int(x) for x in purpdrop[0] if x != "n" ]
    if len(purpdrop) > 0:
        bc = 0
        for b in purpdrop:
            bb = int(b - bc)
            del purp[bb]
            bc += 1
                                    

In [7]:
print("Initializing...")
time_id = 0
pair = []
symbols=mt5.symbols_get()
for s in symbols:
    pair.append(s.name)

levslist = levelgrab(pair)
purp = []

t2 = 0
while True:
    Drates = mt5.copy_rates_from_pos("EURUSD", mt5.TIMEFRAME_M5, 0, 1)
    D = Drates[0]
    t = D[0]
    if t2 != 0:
        if t != t2:
            print("Analyzing Markets...")
            brutus(pair,levslist,time_id)
            time_id += 1
    t2 = t
    time.sleep(1)

Initializing...
Analyzing Markets...
Empty DataFrame
Columns: [x, position, entry, takeprofit, stoploss, CM, G, time_id, spy]
Index: []


C:\Users\User\.conda\envs\EnvC\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


Analyzing Markets...
Empty DataFrame
Columns: [x, position, entry, takeprofit, stoploss, CM, G, time_id, spy]
Index: []


C:\Users\User\.conda\envs\EnvC\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


Analyzing Markets...
Empty DataFrame
Columns: [x, position, entry, takeprofit, stoploss, CM, G, time_id, spy]
Index: []


C:\Users\User\.conda\envs\EnvC\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


KeyboardInterrupt: 